In [ ]:
# create Z2-phase with a hole

In [ ]:
from bloqade import save, load
from bloqade.atom_arrangement import Chain
import numpy as np
import os, time

In [ ]:
from collections import OrderedDict
def invert_keys(D):  # maps g-->0, r -->1  to stick with computer science convention
    # Lambda function to invert the '0's and '1's
    invert_binary = lambda s: s.replace('0', '2').replace('1', '0').replace('2', '1')
    D_inverted = [OrderedDict((invert_binary(k), v) for k, v in d.items()) for d in D]
    return D_inverted


In [ ]:
# Define relevant parameters for the lattice geometry and pulse schedule
n_atoms = 9 #
lattice_const = 6.1

# Define Rabi amplitude and detuning values.
rabi_amplitude = [0.0, 15., 15., 0.0]
delta_glob = [-16, -16, 16, 16]
durations = [0.5, "sweep_time", 0.5]

prog1 = (
    Chain(n_atoms, lattice_const)
    .rydberg.rabi.amplitude.uniform.piecewise_linear(durations, rabi_amplitude)
    .detuning.uniform.piecewise_linear(durations, delta_glob)
)

In [ ]:
job1 = prog1.assign(sweep_time=3.0)
#print(program)
t0=time.time()
emu1 = job1.braket.local_emulator().run(100)
print('run elaT=%.1f sec'%(time.time()-t0))
report1 = emu1.report()

In [ ]:
counts1=invert_keys(report1.counts)   #  is '1'=rydberg
print('Counts1:',counts1)

In [ ]:
# use detunning to suppress 1-state on atom #2 counting form 0 ,the  goal state is: 100010101 

In [ ]:
delta_glob2 = [-16, -16, -16, -16]
delta_local2 = [0., 0.,  32., 32]
atomLab=[i for i in range(n_atoms) ]
atomScale=[1. for i in range(n_atoms) ]; atomScale[2]=0.

prog2 = (
    Chain(n_atoms, lattice_const)
    .rydberg.rabi.amplitude.uniform.piecewise_linear(durations, rabi_amplitude)
    .detuning.uniform.piecewise_linear(durations, delta_glob2)
    .location(atomLab,atomScale).piecewise_linear(durations, values=delta_local2)
)

In [ ]:
print('atoms subject to detune:',atomScale)

In [ ]:
job2 = prog2.assign(sweep_time=3.0)
t0=time.time()
emu2 = job2.braket.local_emulator().run(100)
print('run elaT=%.1f sec'%(time.time()-t0))
report2 = emu2.report()

In [ ]:
counts2=invert_keys(report2.counts)    #  is '1'=rydberg
print('Counts2:',counts2)